In [9]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
tf.keras.backend.clear_session()

In [4]:
import zipfile

In [5]:
zip_path = '/content/data.zip'
extract_to = 'extracted_data/'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

In [11]:
IMAGE_SIZE = (45, 45)
BATCH_SIZE = 32

Generate data from datasets

In [12]:
dataset_dir = "/content/extracted_data/data/extracted_images"
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=10,  # Rotate images by 10 degrees
    width_shift_range=0.1,  # Shift images horizontally by 10%
    height_shift_range=0.1,  # Shift images vertically by 10%
    validation_split=0.2  # Use 20% of the data for validation
)

Load the training test

In [13]:
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=IMAGE_SIZE,
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='categorical',  # Multiclass classification
    subset='training'  # Use 80% of the data for training
)

Found 222324 images belonging to 32 classes.


Load the validation set

In [15]:
validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=IMAGE_SIZE,
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'  # Use 20% of the data for validation
)

Found 55565 images belonging to 32 classes.


Construct the CNN model

In [16]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(45, 45, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(32, activation='softmax')  # 32 output classes for 32 expressions
])

Compile the model

In [17]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

Epoch 1/10
6948/6948 [==============================] - 906s 130ms/step - loss: 0.5118 - accuracy: 0.8479 - val_loss: 0.3413 - val_accuracy: 0.8962
Epoch 2/10
6948/6948 [==============================] - 842s 121ms/step - loss: 0.2730 - accuracy: 0.9139 - val_loss: 0.2413 - val_accuracy: 0.9235
Epoch 3/10
6948/6948 [==============================] - 858s 123ms/step - loss: 0.2353 - accuracy: 0.9249 - val_loss: 0.2004 - val_accuracy: 0.9339
Epoch 4/10
6948/6948 [==============================] - 831s 120ms/step - loss: 0.2134 - accuracy: 0.9316 - val_loss: 0.2070 - val_accuracy: 0.9322
Epoch 5/10
6948/6948 [==============================] - 844s 122ms/step - loss: 0.2005 - accuracy: 0.9351 - val_loss: 0.1942 - val_accuracy: 0.9352
Epoch 6/10
6948/6948 [==============================] - 839s 121ms/step - loss: 0.1893 - accuracy: 0.9384 - val_loss: 0.1735 - val_accuracy: 0.9421
Epoch 7/10
6948/6948 [==============================] - 829s 119ms/step - loss: 0.1831 - accuracy: 0.9393 - val_

Evaluate & save the model

In [18]:
val_loss, val_acc = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_acc}")

model.save('math_expression_recognizer.h5')

1737/1737 [==============================] - 88s 50ms/step - loss: 0.1594 - accuracy: 0.9459
Validation Accuracy: 0.9459012150764465


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
model.save('math_expression_recognizer.keras')

In [20]:
!tensorflowjs_converter --input_format=keras math_expression_recognizer.h5 web_model

2024-09-15 08:55:42.668541: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-15 08:55:42.668659: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-15 08:55:42.676479: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-15 08:55:45.210627: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [19]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, 'web_model2')

In [21]:
!pip install tensorflow==2.15.0
!pip install tensorflow-decision-forests==1.8.1
!pip install tf-keras==2.15.0

#https://stackoverflow.com/a/78501963/24061043
#https://github.com/tensorflow/tfjs/issues/8321

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 999.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.0
    Uninstalling ml-dtypes-0.4.0:
      Successfully uninstalled ml-dtypes-0.4.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard
    Found existing installation

In [6]:
!pip install tensorflowjs

INFO: pip is looking at multiple versions of tensorstore to determine which version is compatible with other requirements. This could take a while.
  Using cached orbax_checkpoint-0.6.3-py3-none-any.whl.metadata (1.8 kB)
INFO: pip is still looking at multiple versions of tensorstore to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: tensorstore
    Found existing installation: tensorstore 0.1.65
    Uninstalling tensorstore-0.1.65:
      Successfully uninstalled tensorstore-0.1.65
  Attempting uninstall: orbax-checkpoint
    Foun

In [21]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.34.2
aiohappyeyeballs                 2.4.0
aiohttp                          3.10.5
aiosignal                        1.3.1
alabaster                        0.7.16
albucore                         0.0.14
albumentations                   1.4.14
altair                           4.2.2
annotated-types                  0.7.0
anyio                            3.7.1
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.18.0
asn1crypto                       1.5.1
astropy                          6.1.3
astropy-iers-data                0.2024.9.11.18.16.39
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            24.2.0
audioread             